Problem 18 from the advent of code calendar 2022.


January 3, 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

np.set_printoptions(linewidth=250)

In [2]:
#Load in the data
with open("19.txt") as f:
    contents = f.readlines()
    
    
def process_line(l):
    """
    Returns the blueprint id and a dictionary where the values are arrays of cost to build the key
    ordered by 
    
        [ore_cost,clay_cost,obsidian_cost,geode_cost]
    """
    s = l.split()
    bid = int(s[1].rstrip(':'))
    ore = int(s[6])
    clay = int(s[12])
    obsidian_ore = int(s[18])
    obsidian_clay = int(s[21])
    geode_ore = int(s[27])
    geode_obsidian = int(s[30])
    
    blueprint = {}
    blueprint['ore'] = np.array([ore,0,0,0])
    blueprint['clay'] = np.array([clay,0,0,0])
    blueprint['obsidian'] = np.array([obsidian_ore, obsidian_clay,0,0])
    blueprint['geode'] = np.array([geode_ore,0,geode_obsidian,0])
    
    return bid, blueprint


#Keeping track of the index of all of our different types of objects
TRANSLATION = {}
TRANSLATION['ore'] = 0
TRANSLATION['clay'] = 1
TRANSLATION['obsidian'] = 2
TRANSLATION['geode'] = 3

In [3]:
"""
Creating a test blueprint from the description to check the code below

Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.


"""

test_dict = {}
test_dict['ore'] = np.array([4,0,0,0])
test_dict['clay'] = np.array([2,0,0,0])
test_dict['obsidian'] = np.array([3, 14,0,0])
test_dict['geode'] = np.array([2,0,7,0])

tid = "t1"


In [4]:
class Robots_and_resources():
    """
    Keeping the robots we have and the resource we have matched up.
    """
    
    def __init__(self, robot_list, resource_list):
        self.robots = robot_list
        self.resources = resource_list
    
    def __repr__(self):
        return 'Robots: {}   Resources: {}'.format(self.robots, self.resources)
    
    def flatten(self):
        return np.append(self.robots,self.resources)
    
    def compare(self,rr):
        """
        Given another robot rr, determine whether rr is better (same robots but more or less resource)
        """
        rval = 0
        if (self.flatten() - rr.flatten()).min() >= 0:
            rval = 1
        elif (self.flatten() - rr.flatten()).max() < 0:
            rval = -1
        return rval
    
    
class Blueprint():
    """
    Tracking traveling through a blueprint
    """
    
    def __init__(self, blueprint_id, blueprint_dict, starting_robots):
        self.id = blueprint_id
        self.blueprints = blueprint_dict
        self.minute_lists = [[starting_robots]] #This is a list where the index indicates the day and the list at that 
                             # index is of the different configurations of robot_and_resources available at that day.

            
    def __repr__(self):
        wstr = 'Blueprint id: {}\n'.format(self.id)
        wstr = wstr + 'Blueprints: {}\n'.format(self.blueprints)
        for i in range(len(self.minute_lists)):
            wstr = wstr + 'After minute {}: \n'.format(i)
            for l in self.minute_lists[i]:
                wstr = wstr + '\t' + str(l) + '\n'
        return wstr
            
    def elapsed_time(self):
        return len(self.minute_lists) - 1
    
    def extend_a_robot(self,robots):
        """
        Input:  
               robots      Robots_and_resources class 
               
        Output:
               List of Robots_and_resources classes that are possible after 1 minute given the blueprints
               
        """
        
        cur_robots = robots.robots.copy()
        cur_resources = robots.resources.copy()
        #Start by putting the case where we don't built any robots into the return list
        nlist = [Robots_and_resources(cur_robots,cur_resources + cur_robots)]
        #Next go through the blueprint and for each one see if it is possible to build that robot.  If it is 
        #possible then add the case where that robot is built to the list.
        for kind in self.blueprints:
            resource_cost = self.blueprints[kind]
            if (cur_resources - resource_cost).min() >= 0:
                #Now checking if we need more of this type of machine
                #print (cur_robots, TRANSLATION[kind])
                if cur_robots[TRANSLATION[kind]] < self.machine_bounds()[TRANSLATION[kind]]:
                 
                    new_resources = cur_resources - resource_cost + cur_robots
                    #Surprising amount of code to increase cur_robots in the right spot to new_robots
                    new_robots = cur_robots.copy()
                    new_robots[TRANSLATION[kind]] += 1
                    nlist.append(Robots_and_resources(new_robots,new_resources))
        return nlist            

    def machine_bounds(self):
        """
        Return how many machines of a given non-geode type are needed to be able to build any machine 
        the following day
        """
        bounds = pd.DataFrame(self.blueprints).max(axis = 1)
        bounds[3] = 100000 #cause we can't have too many geode machines
        return bounds
        
    
    def trim_list(self,tlist):
        """
        At a certain point the possibility list gets *way* too long.  So we need to trim it back.  We compare
        all cases where we have the same number of robots and only keep the ones that maximize the number of 
        resources. Note we are doing this on an external and not an internal list.  Just like the previous 
        function.
        """
        #weights = np.array([1000,100,10,1,0,0,0,0])
        rlist = [x.flatten() for x in tlist]
        #print("Original length: {}".format(len(rlist)))
        rlist = np.unique(np.array(rlist),axis = 0)
        rlist = np.unique(rlist,axis = 0)
        #print("Deduped length: {}".format(len(rlist)))
        #rlist.sort(key = lambda x: (x.flatten()*weights).sum())
        #print(np.array(rlist))
        bustle = rlist
        bust = np.append(bustle[np.where(np.diff(bustle,axis = 0).min(axis = 1) < 0)],[bustle[-1,:]],axis = 0)
        #print("Post trim length: {}".format(bust.shape))
        #print(bust)
        
        
        rlist = [Robots_and_resources(x[:4],x[4:]) for x in bust]
        
        
        
        
        """
        for i in range(len(tlist)):
            rr = tlist[i]
            keep = True
            for j in range(i+1,len(tlist)):
                if tlist[j].compare(rr) >0:
                    keep = False
            if keep:
                rlist.append(rr)
                
        """
        return rlist
    
    
    
    def add_a_minute(self):
        cur_list = self.minute_lists[-1]
        next_list = []
        for rr in cur_list:
            next_list.extend(self.extend_a_robot(rr))
        next_list = self.trim_list(next_list)
        if len(next_list) > 10:
            next_list = self.trim_list(next_list)
        if len(next_list) > 100:
            next_list = self.trim_list(next_list)
        if len(next_list) > 1000:
            next_list = self.trim_list(next_list)
            next_list = self.trim_list(next_list)
        self.minute_lists.append(next_list)
        return None
         
    
    def geodes(self,minute):
        """
        Returns the max number of geodes that could be found in that minute
        """
        assert(minute <= self.elapsed_time())
        tlist = [x.resources[3] for x in self.minute_lists[minute]]
        return max(tlist)
    
#Note that all blueprints start with 1 ore robot and nothing else        
STARTING_STATE = Robots_and_resources(np.array([1,0,0,0]),np.array([0,0,0,0])) 


        

In [5]:
STARTING_STATE

Robots: [1 0 0 0]   Resources: [0 0 0 0]

In [6]:
entry = 0
for line in range(30):
    b = Blueprint(*process_line(contents[line]), STARTING_STATE)
    print("STARTING on blueprint {}".format(b.id))
    
    for i in range(24):
        b.add_a_minute()
        print('Branches: {}\t Elapsed time: {} \t Geodes found: {}'.format(len(b.minute_lists[-1]),b.elapsed_time(),b.geodes(b.elapsed_time())) )   
    
    entry = entry + b.id*b.geodes(24)

print("AND THE ENTRY SHOULD BE (FINGERS CROSSED) {}".format(entry))
    
    

STARTING on blueprint 1
Branches: 1	 Elapsed time: 1 	 Geodes found: 0
Branches: 1	 Elapsed time: 2 	 Geodes found: 0
Branches: 2	 Elapsed time: 3 	 Geodes found: 0
Branches: 2	 Elapsed time: 4 	 Geodes found: 0
Branches: 4	 Elapsed time: 5 	 Geodes found: 0
Branches: 6	 Elapsed time: 6 	 Geodes found: 0
Branches: 10	 Elapsed time: 7 	 Geodes found: 0
Branches: 14	 Elapsed time: 8 	 Geodes found: 0
Branches: 19	 Elapsed time: 9 	 Geodes found: 0
Branches: 23	 Elapsed time: 10 	 Geodes found: 0
Branches: 28	 Elapsed time: 11 	 Geodes found: 0
Branches: 31	 Elapsed time: 12 	 Geodes found: 0
Branches: 34	 Elapsed time: 13 	 Geodes found: 0
Branches: 39	 Elapsed time: 14 	 Geodes found: 0
Branches: 49	 Elapsed time: 15 	 Geodes found: 0
Branches: 64	 Elapsed time: 16 	 Geodes found: 0
Branches: 86	 Elapsed time: 17 	 Geodes found: 0
Branches: 104	 Elapsed time: 18 	 Geodes found: 0
Branches: 145	 Elapsed time: 19 	 Geodes found: 0
Branches: 211	 Elapsed time: 20 	 Geodes found: 0
Branches

Branches: 2856	 Elapsed time: 22 	 Geodes found: 1
Branches: 4711	 Elapsed time: 23 	 Geodes found: 2
Branches: 7888	 Elapsed time: 24 	 Geodes found: 3
STARTING on blueprint 8
Branches: 1	 Elapsed time: 1 	 Geodes found: 0
Branches: 1	 Elapsed time: 2 	 Geodes found: 0
Branches: 1	 Elapsed time: 3 	 Geodes found: 0
Branches: 2	 Elapsed time: 4 	 Geodes found: 0
Branches: 3	 Elapsed time: 5 	 Geodes found: 0
Branches: 4	 Elapsed time: 6 	 Geodes found: 0
Branches: 5	 Elapsed time: 7 	 Geodes found: 0
Branches: 9	 Elapsed time: 8 	 Geodes found: 0
Branches: 14	 Elapsed time: 9 	 Geodes found: 0
Branches: 20	 Elapsed time: 10 	 Geodes found: 0
Branches: 26	 Elapsed time: 11 	 Geodes found: 0
Branches: 41	 Elapsed time: 12 	 Geodes found: 0
Branches: 71	 Elapsed time: 13 	 Geodes found: 0
Branches: 101	 Elapsed time: 14 	 Geodes found: 0
Branches: 146	 Elapsed time: 15 	 Geodes found: 0
Branches: 234	 Elapsed time: 16 	 Geodes found: 0
Branches: 374	 Elapsed time: 17 	 Geodes found: 0
Bra

Branches: 205	 Elapsed time: 20 	 Geodes found: 0
Branches: 303	 Elapsed time: 21 	 Geodes found: 0
Branches: 437	 Elapsed time: 22 	 Geodes found: 0
Branches: 689	 Elapsed time: 23 	 Geodes found: 0
Branches: 1000	 Elapsed time: 24 	 Geodes found: 1
STARTING on blueprint 15
Branches: 1	 Elapsed time: 1 	 Geodes found: 0
Branches: 1	 Elapsed time: 2 	 Geodes found: 0
Branches: 1	 Elapsed time: 3 	 Geodes found: 0
Branches: 1	 Elapsed time: 4 	 Geodes found: 0
Branches: 3	 Elapsed time: 5 	 Geodes found: 0
Branches: 3	 Elapsed time: 6 	 Geodes found: 0
Branches: 3	 Elapsed time: 7 	 Geodes found: 0
Branches: 5	 Elapsed time: 8 	 Geodes found: 0
Branches: 7	 Elapsed time: 9 	 Geodes found: 0
Branches: 11	 Elapsed time: 10 	 Geodes found: 0
Branches: 14	 Elapsed time: 11 	 Geodes found: 0
Branches: 24	 Elapsed time: 12 	 Geodes found: 0
Branches: 29	 Elapsed time: 13 	 Geodes found: 0
Branches: 40	 Elapsed time: 14 	 Geodes found: 0
Branches: 52	 Elapsed time: 15 	 Geodes found: 0
Branche

Branches: 244	 Elapsed time: 18 	 Geodes found: 0
Branches: 357	 Elapsed time: 19 	 Geodes found: 0
Branches: 554	 Elapsed time: 20 	 Geodes found: 0
Branches: 841	 Elapsed time: 21 	 Geodes found: 0
Branches: 1215	 Elapsed time: 22 	 Geodes found: 1
Branches: 1959	 Elapsed time: 23 	 Geodes found: 2
Branches: 3219	 Elapsed time: 24 	 Geodes found: 3
STARTING on blueprint 22
Branches: 1	 Elapsed time: 1 	 Geodes found: 0
Branches: 1	 Elapsed time: 2 	 Geodes found: 0
Branches: 1	 Elapsed time: 3 	 Geodes found: 0
Branches: 2	 Elapsed time: 4 	 Geodes found: 0
Branches: 3	 Elapsed time: 5 	 Geodes found: 0
Branches: 3	 Elapsed time: 6 	 Geodes found: 0
Branches: 5	 Elapsed time: 7 	 Geodes found: 0
Branches: 7	 Elapsed time: 8 	 Geodes found: 0
Branches: 10	 Elapsed time: 9 	 Geodes found: 0
Branches: 15	 Elapsed time: 10 	 Geodes found: 0
Branches: 23	 Elapsed time: 11 	 Geodes found: 0
Branches: 32	 Elapsed time: 12 	 Geodes found: 0
Branches: 43	 Elapsed time: 13 	 Geodes found: 0
Br

Branches: 166	 Elapsed time: 17 	 Geodes found: 0
Branches: 290	 Elapsed time: 18 	 Geodes found: 0
Branches: 458	 Elapsed time: 19 	 Geodes found: 0
Branches: 802	 Elapsed time: 20 	 Geodes found: 0
Branches: 1135	 Elapsed time: 21 	 Geodes found: 0
Branches: 1821	 Elapsed time: 22 	 Geodes found: 0
Branches: 2559	 Elapsed time: 23 	 Geodes found: 0
Branches: 4135	 Elapsed time: 24 	 Geodes found: 1
STARTING on blueprint 29
Branches: 1	 Elapsed time: 1 	 Geodes found: 0
Branches: 1	 Elapsed time: 2 	 Geodes found: 0
Branches: 1	 Elapsed time: 3 	 Geodes found: 0
Branches: 1	 Elapsed time: 4 	 Geodes found: 0
Branches: 3	 Elapsed time: 5 	 Geodes found: 0
Branches: 3	 Elapsed time: 6 	 Geodes found: 0
Branches: 3	 Elapsed time: 7 	 Geodes found: 0
Branches: 5	 Elapsed time: 8 	 Geodes found: 0
Branches: 7	 Elapsed time: 9 	 Geodes found: 0
Branches: 11	 Elapsed time: 10 	 Geodes found: 0
Branches: 14	 Elapsed time: 11 	 Geodes found: 0
Branches: 24	 Elapsed time: 12 	 Geodes found: 0
B

In [7]:
second_entry = 1
for line in range(3):
    b = Blueprint(*process_line(contents[line]), STARTING_STATE)
    print("STARTING on blueprint {}".format(b.id))
    
    for i in range(32):
        b.add_a_minute()
        print('Branches: {}\t Elapsed time: {} \t Geodes found: {}'.format(len(b.minute_lists[-1]),b.elapsed_time(),b.geodes(b.elapsed_time())) )   
    
    second_entry = entry *b.geodes(32)

print("AND THE ENTRY SHOULD BE (FINGERS CROSSED) {}".format(second_entry))
    
    

STARTING on blueprint 1
Branches: 1	 Elapsed time: 1 	 Geodes found: 0
Branches: 1	 Elapsed time: 2 	 Geodes found: 0
Branches: 2	 Elapsed time: 3 	 Geodes found: 0
Branches: 2	 Elapsed time: 4 	 Geodes found: 0
Branches: 4	 Elapsed time: 5 	 Geodes found: 0
Branches: 6	 Elapsed time: 6 	 Geodes found: 0
Branches: 10	 Elapsed time: 7 	 Geodes found: 0
Branches: 14	 Elapsed time: 8 	 Geodes found: 0
Branches: 19	 Elapsed time: 9 	 Geodes found: 0
Branches: 23	 Elapsed time: 10 	 Geodes found: 0
Branches: 28	 Elapsed time: 11 	 Geodes found: 0
Branches: 31	 Elapsed time: 12 	 Geodes found: 0
Branches: 34	 Elapsed time: 13 	 Geodes found: 0
Branches: 39	 Elapsed time: 14 	 Geodes found: 0
Branches: 49	 Elapsed time: 15 	 Geodes found: 0
Branches: 64	 Elapsed time: 16 	 Geodes found: 0
Branches: 86	 Elapsed time: 17 	 Geodes found: 0
Branches: 104	 Elapsed time: 18 	 Geodes found: 0
Branches: 145	 Elapsed time: 19 	 Geodes found: 0
Branches: 211	 Elapsed time: 20 	 Geodes found: 0
Branches

In [8]:
21*10*35

7350